# IIC-3670 NLP UC

! pip install datasets transformers

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import transformers

print(transformers.__version__)

4.29.0


In [4]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

True
NVIDIA RTX A5000
11.7


### Huggingface incluye un helper para debug data. Más utilidades en 
https://huggingface.co/docs/huggingface_hub/main/en/package_reference/utilities

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

## Fine-tuning a DistilGPT2 con Wikipedia

### Vamos a descargar wikitext, ver más datasets en 

https://huggingface.co/docs/datasets/loading

Archivos raw descargados desde

https://cosmo.zip/pub/datasets/wikitext-2-raw/

In [6]:
from datasets import load_dataset


datasets = load_dataset('text', data_files={'train': 'wikitext/wiki.train.raw', 
                                           'validation': 'wikitext/wiki.valid.raw', 
                                           'test': 'wikitext/wiki.test.raw'})


Using custom data configuration default-87494e0b3c88bd8d
Reusing dataset text (/home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


### Vamos a ver alguno ejemplos del dataset

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
    
show_random_elements(datasets["train"])

,text
0,"The music video was co @-@ directed by Ray Kay and Beyoncé for the B 'Day Anthology Video Album , which was released the same month : it was one of eight videos shot in two weeks for the album . The choreography was done by Danielle Polanco and Jonte ' Moaning , who used a 1980 ’ s retro set . Beyoncé explained the concept of the video at MTV : "" It 's probably the most flamboyant video , and the metallic dresses are so beautiful , they added so much color . I had to do a video for this song . Everyone wanted to know what a ' freakum dress ' was , and you can 't really explain it , you have to see it . Everyone has their own version , so we had so many women — of different races , sizes , shapes , ages — because we all have those dresses we pull out when we need to shut it down . """
1,= = Other important information = =
2,
3,
4,"New York State Route 368 ( NY 368 ) was a state highway in Onondaga County , New York , in the United States . It was one of the shortest routes in the county , extending for only 1 @.@ 69 miles ( 2 @.@ 72 km ) between NY 321 and NY 5 in the town of Elbridge . NY 368 was known as Halfway Road for the hamlet it served near its midpoint . The route was assigned in the 1930s and removed in 1980 as part of a highway maintenance swap between the state of New York and Onondaga County ."
5,
6,"Sonate de concert , Op. 47 , for cello and piano – played by Steven Osborne ( piano ) and Alban Gebhart ( cello ) . Recorded 2008 . Hyperion CDA67624 ( 2008 ) ."
7,"After his tour , Dylan returned to New York , but the pressures increased . ABC Television had paid an advance for a TV show . His publisher , Macmillan , was demanding a manuscript of the poem / novel Tarantula . Manager Albert Grossman had scheduled a concert tour for the latter part of the year ."
8,
9,"The burning in 1789 of Christian Murphy , for coining , received practically no attention from the newspapers ( perhaps owing to practical limitations on how much news they could publish across only four pages ) , but it may have been enacted by Sir Benjamin Hammett , a former sheriff of London . Hammett was also an MP , and in 1790 he introduced to Parliament a Bill for Altering the Sentence of Burning Women . He denounced the punishment as "" the savage remains of Norman policy "" which "" disgraced our statutes "" , as "" the practice did the common law "" . He also highlighted how a sheriff who refused to carry out the sentence was liable to prosecution . William Wilberforce and Hammett were not the first men to attempt to end the burning of women . Almost 140 years earlier , during the Interregnum , a group of lawyers and laymen known as the Hale Commission ( after its chairman Matthew Hale ) , was tasked by the House of Commons to take "" into consideration what inconveniences there are in the law "" . Among the proposed reforms was the replacement of burning at the stake with hanging , but , mainly through the objections of various interested parties , none of the commission 's proposals made it into law during the Rump Parliament . Hammett was confident though . He believed that public opinion was on his side and that "" the House would go with him in the cause of humanity "" . The change in execution venues , from Tyburn to Newgate , also attracted criticism . Following Phoebe Harris 's burning in 1786 , as well as questioning the inequality of English law The Times complained about the location of the punishment and its effect on locals :"


### Vamos a trabajar con un modelo pre-entrenado

In [8]:
model_checkpoint = "distilgpt2"

In [9]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(examples["text"])

### Algo muy relevante de transformers es el tokenizer que vamos a usar, el cual debe ser consistente con el usado cuanso se entrenó el LLM. Usamos AutoTokenizer para implementar esta parte.

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

### El wrapper es un tokenizer que usa el autotokenizer, el cual a su vez es consistente con el modelo

In [11]:
tokenizer_wrapper = TokenizerWrapper(tokenizer)

### Ahora tokenizamos los datasets de wikitext

In [12]:
tokenized_datasets = datasets.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ced325a633499540.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-268a3d3b2db002d2.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-96151d5b1878ba97.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-d576064cbfd83386.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-083efea1cc6473cd.arrow


...

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2f0736d8c8cc74f3.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4aafa3555bc66335.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-f7ea51f83a2e377c.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-08ea5c52e5c8e6e1.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4b8933684b4e8df8.arrow


### Fíjese que la máscara de atención viene en 1. Esto es atención global. ¿Por qué?

In [13]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

### Group_texts construye los batches para hacer el fine-tuning

In [14]:
def group_texts(examples):
    block_size = 128
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

### Usamos la función map para aplicar group_texts al dataset tokenizado

In [15]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-8f042679fae47b41.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5ac7876e06f63ef2.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-94ebbaa40b96b3c4.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-41b54c84223caa04.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-824c1796f712e859.arrow


...

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e99d559adf5b1607.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-90c977bbde88d257.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e60c5dffc18f00e0.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e10aeb854d73896b.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1ca19f18869eb84e.arrow


### Un decoder nos permite devolvernos al espacio del texto (desde los IDs)

In [16]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ".  The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

### Vamos a usar AutoModelForCausalLM para cargar el LLM. ¿Por qué ahora usamos causalidad?

In [17]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

### Y ahora usamos el trainer de transformers

In [18]:
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [20]:
trainer.train()

/home/marcelo/.local/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/marcelo/.netrc


Epoch,Training Loss,Validation Loss
1,3.789500,3.687077
2,3.673800,3.665438
3,3.613500,3.663053


TrainOutput(global_step=6969, training_loss=3.7129432611442192, metrics={'train_runtime': 551.6615, 'train_samples_per_second': 101.057, 'train_steps_per_second': 12.633, 'total_flos': 1820879068594176.0, 'train_loss': 3.7129432611442192, 'epoch': 3.0})

### Guardamos los pesos del modelo

In [21]:
trainer.save_model("distilgpt2-finetuned.h5")

### Para luego poder reusarlos. Yo hice fine tuning usando next token prediction, que es reentrenar el base sobre un dataset nuevo (wikitext). Generalmente se hace fine tuning para una downstream task, por ejemplo, clasificación.

In [22]:
from transformers import DistilBertConfig, DistilBertModel

model = DistilBertModel.from_pretrained("distilgpt2-finetuned.h5")

You are using a model of type gpt2 to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilgpt2-finetuned.h5 were not used when initializing DistilBertModel: ['transformer.h.4.ln_1.bias', 'transformer.h.5.attn.c_attn.weight', 'transformer.h.4.mlp.c_fc.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.4.mlp.c_proj.weight', 'transformer.h.2.mlp.c_fc.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.2.attn.c_proj.bias', 'transformer.h.2.attn.c_proj.weight', 'transformer.h.5.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.3.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.5.mlp.c_proj.weight', 'transformer.h.2.mlp.c_fc.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.3.mlp.c_fc.bias', 'transformer.h.0.attn.bias', 'transformer.h.3.ln_2.bias', 'transformer.h.5.attn.c_proj.bias', 'transform

### Vamos a hacer lo mismo pero sobre un transformer encoder

In [23]:
model_checkpoint = "distilroberta-base"

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-268a3d3b2db002d2.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ced325a633499540.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-96151d5b1878ba97.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-d576064cbfd83386.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-083efea1cc6473cd.arrow


...

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2f0736d8c8cc74f3.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-f7ea51f83a2e377c.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4aafa3555bc66335.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-08ea5c52e5c8e6e1.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4b8933684b4e8df8.arrow


In [25]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-8f042679fae47b41.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5ac7876e06f63ef2.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-94ebbaa40b96b3c4.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-41b54c84223caa04.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-824c1796f712e859.arrow


...

Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e8eb833e0fdfb2a1.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e99d559adf5b1607.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-90c977bbde88d257.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e60c5dffc18f00e0.arrow


Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e10aeb854d73896b.arrow
Loading cached processed dataset at /home/marcelo/.cache/huggingface/datasets/text/default-87494e0b3c88bd8d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1ca19f18869eb84e.arrow


### Y usando esos pesos, inicializo un MaskedLM

In [26]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

### Y le hago fine tuning usando wikitext

In [27]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

### Para hacer MLM uso dataCollator, el cual crea las máscaras on the fly

In [28]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

### Y entrenamos

In [30]:
trainer.train()

/home/marcelo/.local/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,5.922500,5.805290
2,5.660300,5.615483
3,5.563200,5.558799


TrainOutput(global_step=6969, training_loss=5.838468449001829, metrics={'train_runtime': 525.9937, 'train_samples_per_second': 105.988, 'train_steps_per_second': 13.249, 'total_flos': 1848383273924352.0, 'train_loss': 5.838468449001829, 'epoch': 3.0})

### Guardamos los pesos

In [34]:
trainer.save_model("distilroberta-finetuned.h5")

In [35]:
from transformers import RobertaModel

model = RobertaModel.from_pretrained("distilroberta-finetuned.h5")

Some weights of the model checkpoint at distilroberta-finetuned.h5 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at distilroberta-finetuned.h5 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### El tokenizer es específico para RoBERTa. Uso ese para leer el modelo finetuned.

In [36]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [37]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0036,  0.1218,  0.0173,  ..., -0.1361, -0.1020, -0.0761],
         [-0.0740,  0.1353,  0.0308,  ..., -0.2017, -0.0584, -0.0205],
         [ 0.0425,  0.1917, -0.0249,  ..., -0.5382,  0.0393,  0.0334],
         ...,
         [ 0.0076,  0.0704, -0.0665,  ...,  0.2193,  0.2206, -0.0945],
         [-0.0229,  0.1350, -0.0236,  ..., -0.1743, -0.1021, -0.0869],
         [ 0.0490,  0.0871,  0.0052,  ..., -0.0363, -0.0795, -0.0254]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.9930e-01,  1.3029e-01,  2.3328e-01,  3.5544e-01,  2.4282e-02,
         -2.0471e-01,  3.7913e-01, -1.7386e-03,  1.5449e-02, -1.2459e-02,
         -1.2383e-01,  2.1325e-01, -6.7104e-01,  4.4308e-01,  2.9050e-01,
          1.1843e-01,  3.5523e-01,  3.2490e-01,  4.1214e-01, -9.5686e-03,
          2.1462e-01,  1.0005e-01, -3.2149e-01,  2.8485e-01,  8.0962e-02,
          5.6077e-01,  1.7002e-01,  1.4226e-01, -2.3036e-01, -2.166

### Using pretrained LLMs (sentence transformers)

In [38]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


### Embeddings para oraciones usando mean_pooling

In [39]:
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0985e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1268e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2530e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0640e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6243e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4717e